In [1]:
import import_ipynb
import data_structures as ds
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import data_structures as ds
from datetime import datetime
import psycopg2
from pyvirtualdisplay import Display
from webdriver_manager.chrome import ChromeDriverManager
import time
from psycopg2.extras import execute_values

importing Jupyter notebook from data_structures.ipynb


In [2]:
def mm_pull_buylist(pullsets, display=False):
    items = []

    browser = webdriver.Chrome(ChromeDriverManager().install())
    print('Driver Loaded')

    browser.get("https://www.miniaturemarket.com/buyback/")
    print(browser)

#     pullsets = pullsets[:10]

    num_sets = len(pullsets)
    print(str(num_sets) + ' in Total')

    for pullset in pullsets:
        print(str(num_sets) + ' sets left!')
        num_sets -= 1

        # if not num_sets in [61, 29, 25]:
        #     continue

        #Drop down the list of sets
        browser.find_element_by_class_name('buyback-category-button').click()

        # Click the set name
        try:
            browser.find_element_by_xpath('//*[@title="' + pullset + '"]').click()
        except Exception as e:
            print(e)
            browser.find_element_by_class_name('buyback-category-button').click()
            continue

        # Click the nonfoil button
        try:
            WebDriverWait(browser, 3).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="filter-mtg_condition"]/span[2]')))
        except Exception as e:
            print(e)

        # get the list of names. If you want foils, do this with class="table-row product-row foil"
        names = browser.find_elements_by_xpath('//*[@class="table-row product-row"]')

        # get the card's core properties
        for name in names:
            line1 = name.text.split('\n')[0]
            line2 = name.text.split('\n')[1]

            try:
                name = line1.split(' - ')[0]
                set = line1.split(' - ')[1].split(' (Near Mint)')[0]
            except Exception as e:
                print(e)
                continue

            # They abbreviate Ikoria's name here ONLY for SOME FUCKING REASON
            if set[:6] == 'Ikoria' and set[-4:] == 'ion)':
                set = 'Ikoria: Lair of Behemoths (Collector Edition)'

            # CM2 is different for SOME REASON
            if set == 'Commander Anthology Volume II':
                set = 'Commander Anthology Volume 2'

            if set != pullset:
                continue

            if line2[0] != '$':
                line2 = line2[line2.find('$'):]

            price = float(line2.lstrip('$').split(' $')[0])


            # if line2[0] == '$':
            #     price = float(line2.lstrip('$').split(' $')[0])
            # else:
            #     price = float(line2.split('$')[1][:-1])
            # print(name)
            # print(set)
            # print(str(price) + '\n')
            items.append([name, set, price])
    browser.quit()

    df = pd.DataFrame(items, columns=['mmname', 'mmset', 'mmbuyprice'])
    if display:
        print(df)
    return df

In [3]:
def store(df):
    conn = psycopg2.connect(host="localhost", port=5432, database="", user="", password="")
    cur = conn.cursor()

    df['uuid'] = 'temp'
    
    c_get_u_mistakes = 0
    for index, row in df.iterrows():
        try:
            uuid = ds.get_uuid(cur, row['mmname'], row['mmset'], 'mm')
            df.at[index, 'uuid'] = uuid
        except Exception as e:
            c_get_u_mistakes += 1
    print('Getting uuid mistakes: ' + str(c_get_u_mistakes))

    df = df.loc[df['uuid'] != 'temp']
    df['mmdate'] = datetime.now()
    t_df = tuple(df[['uuid', 'mmdate', 'mmbuyprice']].itertuples(index=False, name=None))
    execute_values(cur, "INSERT INTO mmbuydata (uuid, mmdate, mmbuyprice) VALUES %s", t_df)
    conn.commit()

    conn.close()
    cur.close()

In [4]:
def pull_and_store():
    display = Display(visible=0, size=(2000, 800))  
    display.start()
    
    conn = psycopg2.connect(host="localhost", port=5432, database="", user="", password="")
    cur = conn.cursor()
    SQL = 'SELECT mmset FROM cardinfo;'
    sets = pd.read_sql(SQL, conn)
    conn.close()
    cur.close()
    
    sets.drop_duplicates(inplace=True, ignore_index=True)
    sets = sets['mmset']
    
    df = mm_pull_buylist(sets, display=True)
    
    display.stop()
    
    store(df)

In [5]:
def main():
    start = time.time()
    pull_and_store()
    end = time.time()
    print(end - start)

In [6]:
if __name__ == '__main__':
    main()

[WDM] - Current google-chrome version is 84.0.4147
[WDM] - Get LATEST driver version for 84.0.4147
[WDM] - Driver [/home/cole/.wdm/drivers/chromedriver/linux64/84.0.4147.30/chromedriver] found in cache


 
Driver Loaded
<selenium.webdriver.chrome.webdriver.WebDriver (session="e446ded79f563ad572f5ccfe99ef896b")>
471 in Total
471 sets left!
Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@title="10th Edition"]"}
  (Session info: chrome=84.0.4147.125)

470 sets left!
Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@title="Avacyn Restored Promos"]"}
  (Session info: chrome=84.0.4147.125)

469 sets left!
Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@title="Unstable"]"}
  (Session info: chrome=84.0.4147.125)

468 sets left!
Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@title="Core Set 2021"]"}
  (Session info: chrome=84.0.4147.125)

467 sets left!
Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@title="Battlebond Promos"]"}
  (Session info: chrome=84.0.4147.125)

466 sets left!
Message: no such elem